In [6]:
!pip3 install folium

    100% |████████████████████████████████| 92kB 10.9MB/s ta 0:00:01
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import folium
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

print("Packages Imported")

crimeDataCsvPath = '/home/ec2-user/SageMaker/wsu-computer-science/CS-483_Big-Data-Analytics-Capstone_2020-Spring/code/common/python/output/WorcesterData_03-19.csv'
crimes = pd.read_csv(crimeDataCsvPath,error_bad_lines=False)

# Clean up duplicate information if it exists
print('Dataset Shape before drop_duplicate : ', crimes.shape)
crimes.drop_duplicates(subset=['P_INCID_NO'], inplace=True)
print('Dataset Shape after drop_duplicate: ', crimes.shape)

# Use line below to drop columns not needed, optional
# crimes.drop(['Year', 'P_INCID_NO'], inplace=True, axis=1)

# display first three rows to confirm we have what we expect
crimes.head(3)
#crimes.shape
#crimes.info()

# Below uses seconds (we dont have seconds) & capital Y instead of lowercase for last two digits of year
#crimes.ACTUAL_DTE = pd.to_datetime(crimes.ACTUAL_DTE, format='%m/%d/%Y %I:%M:%S %p')
#crimes.ACTUAL_DTE = pd.to_datetime(crimes.ACTUAL_DTE, format='%m/%d/%y %I:%M %p')
crimes.ACTUAL_DTE = pd.to_datetime(crimes.ACTUAL_DTE, format='%m/%d/%Y %H:%M:S', infer_datetime_format=True)

# Set the index to the date time (FOR MAPS ONLY)
#crimes = crimes.set_index(['ACTUAL_DTE'])

# Set the categorical data for pandas
crimes['PRIMARY_STREET_NAME']        	= pd.Categorical(crimes['PRIMARY_STREET_NAME'])
crimes['PRIMARY_CITY'] 					= pd.Categorical(crimes['PRIMARY_CITY'])
crimes['PRIMARY_STATE_CD']          	= pd.Categorical(crimes['PRIMARY_STATE_CD'])
crimes['PRIMARY_ZIP_CD']                = pd.Categorical(crimes['PRIMARY_ZIP_CD'])
crimes['PRIMARY_ADDR_NO']        	    = pd.Categorical(crimes['PRIMARY_ADDR_NO'])
crimes['IBR_TYPE']          			= pd.Categorical(crimes['IBR_TYPE'])
crimes['IBR_TYPE_DESC']          		= pd.Categorical(crimes['IBR_TYPE_DESC'])

# Set the integer values (float by default)
crimes                                  = crimes.astype({'PRIMARY_STREET_NUM': 'Int32'})

# Done
print('You are ready to use the data')

Packages Imported
Dataset Shape before drop_duplicate :  (264342, 14)
Dataset Shape after drop_duplicate:  (231999, 14)
You are ready to use the data


In [26]:
# --------------------------------------------------
# ----- Create a copy of the crimes dataset
# --------------------------------------------------
ranked_crimes = crimes.copy()

In [27]:
# --------------------------------------------------
# ----- Group the incidents into bins
# --------------------------------------------------
chunk_size = 0.005
to_bin = lambda x: np.floor(x / chunk_size) * chunk_size
ranked_crimes["latbin"] = ranked_crimes.LATITUDE.map(to_bin)
ranked_crimes["lonbin"] = ranked_crimes.LONGITUDE.map(to_bin)
# groups = ranked_crimes.groupby(("latbin", "lonbin"))

In [28]:
# --------------------------------------------------
# ----- Determine the different types of incidents
# --------------------------------------------------
incident_values = ranked_crimes[["IBR_TYPE", "IBR_TYPE_DESC"]].values.ravel()
unique_incident_values =  pd.unique(incident_values)
print(unique_incident_values)

['23G' 'Theft of Motor Vehicle Parts' 'ZZZ' 'Non-NIBRS Offense' '23H'
 'All Other Thefts' '40A' 'Prostitution' '35A' 'Drug/Narcotic Violations'
 '240' 'Motor Vehicle Theft' '23F' 'Theft from Motor Vehicle' '120'
 'Robbery' '290' 'Destruction/Damage/Vandalism of Property' '23D'
 'Theft from Building' '13C' 'Assault - Intimidation' '13A'
 'Aggravated Assault' '220' 'Burglary/Breaking & Entering' '11D'
 'Forcible Fondling' '26C' 'Impersonation' '13B' 'Simple Assault' '23C'
 'Shoplifting' '520' 'Weapon Law Violation' '90C' 'Disorderly Conduct'
 '90Z' 'All Other Offenses' '26E' 'Wire Fraud' '90J'
 'Trespass of Real Property' '90D' 'Driving Under the Influence' '90I'
 'Runaway' '250' 'Counterfeiting/Forgery' '26B'
 'Credit Card/Automatic Teller Fraud' '90G' 'Liquor Law Violation' '35B'
 'Drug Equipment Violations' '90F' 'Family Offenses, Nonviolent' '90N'
 'Resisting Arrest' '200' 'Arson' '11A' 'Forcible Rape' '36B'
 'Statutory Rape (Nonforcible Sex Offense)' '11B' 'Forcible Sodomy' '26A'
 '

In [29]:
# Turn it into a new dataframe to join with
# Assign weight values to different ranked_crimes to represent the danger level. These weight values are arbitrary, relative to statistics, I chose them myself.
# Key                       Value
# Violent Incident          9
# Property Incident         7
# Non-Violent Incident      5
# White-Collar Incident     3
# Low-Risk Incident         2
# Other / Unkown Incident   1


# import the StrinIO function 
# from io module 
from io import StringIO 

# let's read the data using the Pandas read_csv() function 

weightedIncidentsString = StringIO("""
INCIDENT_CODE;INCIDENT_DESC;INCIDENT_WEIGHT;
23G;Theft of Motor Vehicle Parts;7;
ZZZ;Non-NIBRS Offense;1;
23H;All Other Thefts;7;
40A;Prostitution;5;
35A;Drug/Narcotic Violations;5;
240;Motor Vehicle Theft;7;
23F;Theft from Motor Vehicle;7;
120;Robbery;7;
290;Destruction/Damage/Vandalism of Property;7;
23D;Theft from Building;7;
13C;Assault - Intimidation;9;
13A;Aggravated Assault;9;
220;Burglary/Breaking & Entering;7;
11D;Forcible Fondling;9;
26C;Impersonation;3;
13B;Simple Assault;9;
23C;Shoplifting;7;
520;Weapon Law Violation;5;
90C;Disorderly Conduct;5;
90Z;All Other Offenses;1;
26E;Wire Fraud;3;
90J;Trespass of Real Property;7;
90D;Driving Under the Influence;5;
90I;Runaway;2;
250;Counterfeiting/Forgery;3;
26B;Credit Card/Automatic Teller Fraud;3;
90G;Liquor Law Violation;3;
35B;Drug Equipment Violations;5;
90F;Family Offenses, Nonviolent;5;
90N;Resisting Arrest;9;
200;Arson;7;
11A;Forcible Rape;9;
36B;Statutory Rape (Nonforcible Sex Offense);2;
11B;Forcible Sodomy;9;
26A;False Pretense/Swindle/Confidence Game;3;
09A;Murder & Nonnegligent Manslaughter;9;
23E;Theft from Coin-operated Machine;7;
280;Stolen Property Offenses;7;
100;Kidnapping/Abduction;9;
270;Embezzlement;3;
90M;Other Agencies (Mental Health);1;
23B;Purse Snatching;9;
370;Pornography/Obscene Material;2;
11C;Sexual Assault with an Object;9;
36A;Incest (Nonforcible Sex Offense);2;
90P;Contributing to the Delinquency of a Minor;2;
23A;Pocket Picking;7;
90E;Drunkenness;5;
210;Extortion/Blackmail;3;
90A;Bad Checks;3;
39C;Gambling Equipment Violations;3;
90L;Truancy;2;
90K;Incorrigible;3;
VOI;VOI (Added during conversion);9;
40B;Assisting or Promoting Prostitution;5;
26F;Identity Theft;3;
720;Animal Cruelty;5;
26G;Hacking/Computer Invasion;3;
39B;Operating/Promoting/Assisting Gambling;3;
64A;Human Trafficking, Commercial Sex Acts;5;
64B;Human Trafficking, Involuntary Servitude;5;
90B;Curfew/Loitering/Vagrancy;2;
26D;Welfare Fraud;3;
Zx;Unknown;1;
""")

weighted_incidents = pd.read_csv(weightedIncidentsString, sep =";") 

weighted_incidents['INCIDENT_CODE']          			= pd.Categorical(weighted_incidents['INCIDENT_CODE'])
weighted_incidents['INCIDENT_DESC']          		= pd.Categorical(weighted_incidents['INCIDENT_DESC'])


In [30]:
# --------------------------------------------------
# ----- Join incidents on the weighted incidents data
# --------------------------------------------------
# reference https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/
ranked_crimes = pd.merge(ranked_crimes, weighted_incidents[["INCIDENT_CODE","INCIDENT_WEIGHT"]], left_on="IBR_TYPE", right_on="INCIDENT_CODE", how="left")


In [31]:
# --------------------------------------------------
# ----- Determine the bin rank by year
# --------------------------------------------------
rankedBins_year = ranked_crimes.groupby(['Year','latbin','lonbin'], as_index=False).agg({"INCIDENT_WEIGHT": "sum"})
rankedBins_year = rankedBins_year.rename(columns={"INCIDENT_WEIGHT": "WEIGHTED_INCIDENT_SCORE"})

# > pip3 install sklearn
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
rankedBins_year['RANK'] = scaler.fit_transform(rankedBins_year[['WEIGHTED_INCIDENT_SCORE']])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [32]:
ranked_crimes.head(3)

,Unnamed: 0,Year,P_INCID_NO,ACTUAL_DTE,PRIMARY_STREET_NUM,PRIMARY_STREET_NAME,PRIMARY_CITY,PRIMARY_STATE_CD,PRIMARY_ZIP_CD,PRIMARY_ADDR_NO,LATITUDE,LONGITUDE,IBR_TYPE,IBR_TYPE_DESC,latbin,lonbin,INCIDENT_CODE,INCIDENT_WEIGHT
0,0,2003,2003000000001,2003-01-08 08:37:00,25,FLORENCE ST,Worcester,MA,1610.0,2.001000e+12,42.254770,-71.821817,23G,Theft of Motor Vehicle Parts,42.25,-71.825,23G,7
1,1,2003,2003000000005,2003-01-08 08:47:00,2,VETERANS AV,Worcester,MA,1603.0,2.002000e+12,42.250486,-71.833306,ZZZ,Non-NIBRS Offense,42.25,-71.835,ZZZ,1
2,2,2003,2003000000007,2003-01-08 08:48:00,1,ASHLAND ST,Worcester,MA,1609.0,2.001000e+12,42.263982,-71.807259,23H,All Other Thefts,42.26,-71.810,23H,7


In [33]:
# --------------------------------------------------
# ----- Determine the bin rank by day
# --------------------------------------------------

rankedBins_day = ranked_crimes.groupby([ranked_crimes.ACTUAL_DTE.dt.year,ranked_crimes.ACTUAL_DTE.dt.month,ranked_crimes.ACTUAL_DTE.dt.day,'latbin','lonbin'], as_index=True).agg({"INCIDENT_WEIGHT": "sum"})
rankedBins_day = rankedBins_day.rename(columns={"INCIDENT_WEIGHT": "WEIGHTED_INCIDENT_SCORE"})

# > pip3 install sklearn
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
rankedBins_day['RANK'] = scaler.fit_transform(rankedBins_day[['WEIGHTED_INCIDENT_SCORE']])


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [34]:
# --------------------------------------------------
# ----- Determine the bin rank by month
# --------------------------------------------------

rankedBins_month = ranked_crimes.groupby([ranked_crimes.ACTUAL_DTE.dt.year,ranked_crimes.ACTUAL_DTE.dt.month,'latbin','lonbin'], as_index=True).agg({"INCIDENT_WEIGHT": "sum"})
rankedBins_month = rankedBins_month.rename(columns={"INCIDENT_WEIGHT": "WEIGHTED_INCIDENT_SCORE"})

# > pip3 install sklearn
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
rankedBins_month['RANK'] = scaler.fit_transform(rankedBins_month[['WEIGHTED_INCIDENT_SCORE']])


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [39]:
# --------------------------------------------------
# ----- Save the data
# --------------------------------------------------

dataPath = '/home/ec2-user/SageMaker/wsu-computer-science/CS-383_Cloud-Computing_2020-Spring/area-ranking/output/'
rankedBins_year.to_csv(dataPath + 'WorcesterData_BinRanked_Year.csv')
rankedBins_month.to_csv(dataPath + 'WorcesterData_BinRanked_Month.csv')
rankedBins_day.to_csv(dataPath + 'WorcesterData_BinRanked_Day.csv')